In [ ]:
import zipfile
import os

zip_file_name = "Data.zip"
extract_dir = "dataset"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"All videos have been extracted to '{extract_dir}/'")


All videos have been extracted to 'dataset/'


In [ ]:
import os
import shutil
import glob
import re

source_dir = '/content/dataset/standardised'
organized_dir = 'organized_dataset'
os.makedirs(organized_dir, exist_ok=True)

pattern = re.compile(r'g\d+', re.IGNORECASE)
video_files = glob.glob(f'{source_dir}/**/*.*', recursive=True)
class_labels = set()
for video_path in video_files:
    filename = os.path.basename(video_path)
    match = pattern.search(filename)
    if match:
        class_labels.add(match.group().lower())
class_labels = list(class_labels)[:5]

print("Detected class labels:", class_labels)
for video_path in video_files:
    filename = os.path.basename(video_path)
    match = pattern.search(filename)

    if match:
        label = match.group().lower()
        if label in class_labels:
            class_folder = os.path.join(organized_dir, label)
            os.makedirs(class_folder, exist_ok=True)
            shutil.move(video_path, os.path.join(class_folder, filename))

print("✅ Videos segregated into folders:", class_labels)


Detected class labels: ['g2', 'g3', 'g4', 'g11', 'g1']
✅ Videos segregated into folders: ['g2', 'g3', 'g4', 'g11', 'g1']


In [ ]:
import os

organized_dir = 'organized_dataset'
class_counts = {}

for class_name in os.listdir(organized_dir):
    class_path = os.path.join(organized_dir, class_name)
    if os.path.isdir(class_path):
        num_files = len([
            f for f in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, f))
        ])
        class_counts[class_name] = num_files

for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} videos")

total_videos = sum(class_counts.values())
print(f"\nTotal videos counted: {total_videos}")


Class 'g11': 182 videos
Class 'g3': 452 videos
Class 'g1': 142 videos
Class 'g2': 464 videos
Class 'g4': 404 videos

Total videos counted: 1644


In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class VideoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_paths, labels, batch_size=4, frames_per_clip=16, img_size=(112, 112), shuffle=True):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.frames_per_clip = frames_per_clip
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.video_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        X, y = self.__data_generation(batch_paths, batch_labels)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.video_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            self.video_paths = [self.video_paths[i] for i in self.indexes]
            self.labels = [self.labels[i] for i in self.indexes]

    def __data_generation(self, video_paths, labels):
        X = []
        y = []

        for video_path, label in zip(video_paths, labels):
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames == 0:
                frames = [np.zeros((*self.img_size, 1), dtype=np.uint8) for _ in range(self.frames_per_clip)]
            else:
                frame_idxs = np.linspace(0, total_frames - 1, self.frames_per_clip).astype(int)
                frames = []
                for idx in frame_idxs:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if ret:
                        frame = cv2.resize(frame, self.img_size)
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                        frame = np.expand_dims(frame, axis=-1)
                        frames.append(frame)
                    else:
                        frames.append(np.zeros((*self.img_size, 1), dtype=np.uint8))
            cap.release()

            clip = np.array(frames) / 255.0  # Normalize to [0,1]
            X.append(clip)
            y.append(label)

        X = np.array(X)
        y = tf.keras.utils.to_categorical(y, num_classes=len(set(self.labels)))
        return X, y

def collect_paths_and_labels(base_path):
    video_paths = []
    labels = []
    class_names = sorted(os.listdir(base_path))
    le = LabelEncoder()
    le.fit(class_names)

    for label_name in class_names:
        class_dir = os.path.join(base_path, label_name)
        if not os.path.isdir(class_dir):
            continue
        for fname in os.listdir(class_dir):
            if fname.lower().endswith(('.mp4', '.avi', '.mov')):
                video_paths.append(os.path.join(class_dir, fname))
                labels.append(label_name)

    labels_encoded = le.transform(labels)
    return video_paths, labels_encoded, le
dataset_path = "Dataset"

video_paths, video_labels, label_encoder = collect_paths_and_labels(dataset_path)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    video_paths, video_labels, test_size=0.2, stratify=video_labels, random_state=42)

print(f"Number of training videos: {len(train_paths)}")
print(f"Number of validation videos: {len(val_paths)}")

train_generator = VideoDataGenerator(train_paths, train_labels, batch_size=4)
val_generator = VideoDataGenerator(val_paths, val_labels, batch_size=4)

Number of training videos: 1315
Number of validation videos: 329


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_conv3d_model_with_bn(input_shape=(16, 112, 112, 1), num_classes=5):
    model = models.Sequential()

    model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    model.add(layers.Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    model.add(layers.Flatten())

    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

num_classes = 5
input_shape = (16, 112, 112, 1)

model = build_conv3d_model_with_bn(input_shape=input_shape, num_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 16, 112, 112,   │           896 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 16, 112, 112,   │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 16, 112, 112,   │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 16, 56, 56, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 16, 56, 56, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 56, 56, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 16, 56, 56, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 8, 28, 28, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 8, 28, 28, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 8, 28, 28, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 8, 28, 28, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 4, 14, 14, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 26,003,909 (99.20 MB)

 Trainable params: 26,002,693 (99.19 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [ ]:
model.fit(train_generator,epochs=10,validation_data=val_generator)

Epoch 1/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 352s 1s/step - accuracy: 0.8485 - loss: 0.4424 - val_accuracy: 0.8997 - val_loss: 0.2756
Epoch 2/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 349s 1s/step - accuracy: 0.8432 - loss: 0.4857 - val_accuracy: 0.9392 - val_loss: 0.2270
Epoch 3/10
